In [1]:
#划分区域
from numpy import *
import numpy as np
import pandas as pd
import math as Math
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
import time
import datetime

def compute_time_interval(start, end):
   
    start = datetime.datetime.fromtimestamp(start / 1000.0)
    end = datetime.datetime.fromtimestamp(end / 1000.0)
    
    # 相减得到秒数
    seconds = (end- start).seconds
    
    return seconds

In [3]:
rc = 6378137
rj = 6356725
from math import atan, cos, asin, sqrt, pow, pi, sin
def rad(d):
    return d * math.pi / 180.0

def azimuth(pt_a, pt_b):
    lon_a, lat_a = pt_a
    lon_b, lat_b = pt_b
    rlon_a, rlat_a = rad(lon_a), rad(lat_a)
    rlon_b, rlat_b = rad(lon_b), rad(lat_b)
    ec=rj+(rc-rj)*(90.-lat_a)/90.
    ed=ec*cos(rlat_a)

    dx = (rlon_b - rlon_a) * ec
    dy = (rlat_b - rlat_a) * ed
    if dy == 0:
        angle = 90. 
    else:
        angle = atan(abs(dx / dy)) * 180.0 / pi
    dlon = lon_b - lon_a
    dlat = lat_b - lat_a
    if dlon > 0 and dlat <= 0:
        angle = (90. - angle) + 90
    elif dlon <= 0 and dlat < 0:
        angle = angle + 180 
    elif dlon < 0 and dlat >= 0:
        angle = (90. - angle) + 270 
    return angle

def distance(true_pt, pred_pt):
    lat1 = float(true_pt[1])
    lng1 = float(true_pt[0])
    lat2 = float(pred_pt[1])
    lng2 = float(pred_pt[0])
    radLat1 = rad(lat1)
    radLat2 = rad(lat2)
    a = radLat1 - radLat2
    b = rad(lng1) - rad(lng2)
    s = 2 * Math.asin(Math.sqrt(Math.pow(Math.sin(a/2),2) +
    Math.cos(radLat1)*Math.cos(radLat2)*Math.pow(Math.sin(b/2),2)))
    s = s * 6378.137
    s = round(s * 10000) / 10
    return s

def sq(x):
    return x*x

In [4]:
col_name_new = [
    #'Num_connected',
    'TrajID',
    'ECGI_1',
    'Dbm_1',
    'Freq_1',
    'CellID_1',
    'ECGI_2',
    'Dbm_2',
    'Freq_2',
    'CellID_2',
    'ECGI_3',
    'Dbm_3',
    'Freq_3',
    'CellID_3',
    'ECGI_4',
    'Dbm_4',
    'Freq_4',
    'CellID_4',
    'ECGI_5',
    'Dbm_5',
    'Freq_5',
    'CellID_5',
    'ECGI_6',
    'Dbm_6',
    'Freq_6',
    'CellID_6',
    'ECGI_7',
    'Dbm_7',
    'Freq_7',
    'CellID_7',
    #'RSSI_6',
]


In [5]:
col_name_rf = [
    'ECGI_1',
    'Dbm_1',
    'Freq_1',
    'CellID_1',
    'ECGI_2',
    'Dbm_2',
    'Freq_2',
    'CellID_2',
    'ECGI_3',
    'Dbm_3',
    'Freq_3',
    'CellID_3',
    'ECGI_4',
    'Dbm_4',
    'Freq_4',
    'CellID_4',
    'ECGI_5',
    'Dbm_5',
    'Freq_5',
    'CellID_5',
    'ECGI_6',
    'Dbm_6',
    'Freq_6',
    'CellID_6',
    'ECGI_7',
    'Dbm_7',
    'Freq_7',
    'CellID_7',
    'Lon','Lat','Lon2','Lat2','Lon3','Lat3','Lon4','Lat4','Lon5','Lat5','Lon6','Lat6','Lon7','Lat7'
]


In [6]:
def merge_2g_engpara():
    eng_para = pd.read_csv('huawei_big_data_gongcan_4g.csv', encoding='utf-8')
    eng_para = eng_para[['ECGI', 'PCI', '1F_UARFCN',u'经度',u'维度']]
    eng_para = eng_para.drop_duplicates()
    eng_para.rename(columns={u'经度': 'Lon', u'维度': 'Lat'}, inplace=True)
    eng_para['BSID'] = range(len(eng_para))
    eng_para['BSID'] = eng_para['BSID'].map(lambda x: x + 1)
    return eng_para

def make_rf_dataset(data, eng_para):
    for i in range(1, 8):
        data = data.merge(eng_para, left_on=['ECGI_%d' % i, 'Freq_%d' % i, 'CellID_%d' % i], right_on=['ECGI','1F_UARFCN','PCI'], 
                          how='left', suffixes=('', '%d' % i))
        temp=data['CellID_%d'% i].tolist()
        new=list()
        for item in temp:
            if math.isnan(item):
                new.append(0)
            elif int(item)<=0:
                new.append(0)
            else:
                new.append(item)
        data['CellID_%d' % i]=new
    data = data.fillna(-999.)
    #print data.columns
    
    feature = data[col_name_new+['MRTime','Timestamp','BSID','BSID2','BSID3','BSID4','BSID5','BSID6','BSID7','Longitude', 'Latitude',
                                 'Lon','Lat','Lon2','Lat2','Lon3','Lat3','Lon4','Lat4','Lon5','Lat5','Lon6','Lat6','Lon7','Lat7']]
   
    
    subset=[u'Longitude', u'Latitude', 
       u'ECGI_1', u'CellID_1',u'Freq_1',
       u'ECGI_2', u'CellID_2',u'Freq_2',
       u'ECGI_3', u'CellID_3',u'Freq_3',
       u'ECGI_4', u'CellID_4',u'Freq_4',
       u'ECGI_5', u'CellID_5',u'Freq_5',
       u'ECGI_6', u'CellID_6',u'Freq_6',
       u'ECGI_7', u'CellID_7',u'Freq_7',
       ]
    #feature=feature.drop_duplicates(subset=subset) 
    label = feature[['Longitude', 'Latitude']]
    feature= feature.drop(['Longitude', 'Latitude'],axis=1)
    
    return feature, label

#eng_para = merge_2g_engpara()
eng_para =merge_2g_engpara()

In [7]:
def conf_model_label(error):
    conf_l=list()
   
    for t in error:
        if t<=50:
            conf_l.append(1)
        else:
            conf_l.append(0)
    
    return conf_l

In [8]:
data_2g=pd.read_csv("new_school.csv")
data_2g = data_2g.drop(data_2g[data_2g['sub_area']==9].index)
data_2g = data_2g.groupby('sub_area')

In [9]:
from sklearn.cross_validation import train_test_split

train, label, tr_feature_r, te_feature_r, tr_label_, te_label_ = [[]]*9, [[]]*9, [[]]*9, [[]]*9, [[]]*9, [[]]*9

for name, group in data_2g:
    i = int(name)
    train[i], label[i] = make_rf_dataset(group, eng_para)
    tr_feature_r[i], te_feature_r[i], tr_label_[i], te_label_[i] = train_test_split(train[i], label[i], test_size=0.4,random_state=50)

/usr/local/lib/python2.7/dist-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [10]:
def conf_pre(data):
    data=data.iloc[:,1:]
    label=data[['Longitude','Latitude']]
    data=data.drop(['Longitude', 'Latitude'],axis=1)
    
    return data, label

In [11]:
con_tr_feature, con_te_feature, con_tr_p, con_te_p = [[]]*9, [[]]*9, [[]]*9, [[]]*9

for i in range(9):
    con_tr_feature[i], con_te_feature[i], con_tr_p[i], con_te_p[i] = train_test_split(te_feature_r[i], te_label_[i], test_size=0.4)

In [12]:
from sklearn.ensemble import RandomForestRegressor,RandomForestClassifier
from sklearn.tree import DecisionTreeRegressor

In [13]:
#con_tr_feature.to_csv("2g/conf_tr_jd2g.csv")
#con_te_feature.to_csv("2g/conf_te_jd2g.csv")
#tr_feature_r.to_csv("2g/total_conf_tr_jd2g.csv")

In [14]:
import grid

rg, tr_label_g, con_tr_j, con_te_j = [[]]*9, [[]]*9, [[]]*9, [[]]*9

for i in range(9):
    rg[i] = grid.RoadGrid(np.vstack((tr_label_[i].values, te_label_[i].values)),100)
    tr_label_g[i] = rg[i].transform(tr_label_[i].values, False) # label所在的grid
    #rint tr_label_
    con_tr_j[i] = rg[i].transform(con_tr_p[i].values, False)
    con_te_j[i] = rg[i].transform(con_te_p[i].values, False)


## CCR

In [78]:
pred_tr, tr_pred1, tr_con_pred, te_con_pred = [[]]*9, [[]]*9, [[]]*9, [[]]*9
error_tr, error_te = [], []

for i in range(9):
    est=RandomForestClassifier( n_jobs=-1,
        n_estimators =50,
        max_features='sqrt'
    ).fit(tr_feature_r[i][col_name_rf].values, tr_label_g[i])

    pred_tr[i]=est.predict(tr_feature_r[i][col_name_rf].values)
    tr_pred1[i] = np.array([rg[i].grid_center[idx] for idx in pred_tr[i]])
    error_tr = [distance(pt1, pt2) for pt1, pt2 in zip(tr_pred1[i], tr_label_[i].values)]

    pred_con_tr=est.predict(con_tr_feature[i][col_name_rf].values)
    pred_con_te=est.predict(con_te_feature[i][col_name_rf].values)
    tr_con_pred[i] = np.array([rg[i].grid_center[idx] for idx in pred_con_tr])
    te_con_pred[i] = np.array([rg[i].grid_center[idx] for idx in pred_con_te])
    error_con_tr = [distance(pt1, pt2) for pt1, pt2 in zip(tr_con_pred[i], con_tr_p[i].values)]
    error_con_te = [distance(pt1, pt2) for pt1, pt2 in zip(te_con_pred[i], con_te_p[i].values)]
    error_tr += error_con_tr
    error_te += error_con_te

In [79]:
error_te = sorted(error_te)
np.median(error_te), np.mean(error_te), error_te[int(len(error_te) * 0.90)]

(47.600000000000001, 90.341951933519056, 212.7)

In [17]:
from scipy.sparse import csc_matrix

In [18]:
def feature_engineer(feature, pred, timestamp):
    add_feature = []
    timestamp_new=np.array(timestamp)
    for i in range(0, len(pred)):
        if i == 0:
            last_pt = pred[i]
            last_time = timestamp_new[i]
        else:
            last_pt = pred[i-1]
            last_time = timestamp_new[i-1]
        if i == len(pred)-1:
            next_pt = pred[i]
            next_time = timestamp_new[i]
        else:
            next_pt = pred[i+1]
            next_time = timestamp_new[i+1]
        sub_add_feature = []
        sub_add_feature.append(distance(last_pt, pred[i]))
        sub_add_feature.append(distance(next_pt, pred[i]))
        sub_add_feature.append(azimuth(last_pt, pred[i]))
        sub_add_feature.append(azimuth(pred[i], next_pt))
        sub_add_feature.append(timestamp_new[i]-last_time if timestamp_new[i]-last_time > 0 else 0)
        sub_add_feature.append(next_time-timestamp_new[i] if next_time-timestamp_new[i] > 0 else 0)
        sub_add_feature.append(sub_add_feature[0] / sub_add_feature[4] if sub_add_feature[4] > 0 else 0)
        sub_add_feature.append(sub_add_feature[1] / sub_add_feature[5] if sub_add_feature[5] > 0 else 0)
        sub_add_feature.append(last_pt[0])
        sub_add_feature.append(last_pt[1])
        sub_add_feature.append(next_pt[0])
        sub_add_feature.append(next_pt[1])
        sub_add_feature.append(pred[i][0])
        sub_add_feature.append(pred[i][1])
        #sub_add_feature.append(grid[i])
        add_feature.append(sub_add_feature)
    add_feature = np.asarray(add_feature)
    features = np.array(np.hstack((feature, add_feature)), dtype=float)
    feature = csc_matrix(features)
    return feature

In [62]:
tr_pred, te_pred, pred_con_tr, pred_con_te, error_tr, error_te = [[]]*9, [[]]*9, [[]]*9, [[]]*9, [[]]*9, [[]]*9

for i in range(9):
    feature_tr = feature_engineer(tr_feature_r[i][col_name_rf], tr_pred1[i], tr_feature_r[i]['Timestamp'])
    feature_con_tr = feature_engineer(con_tr_feature[i][col_name_rf], tr_con_pred[i], con_tr_feature[i]['Timestamp'])
    feature_con_te = feature_engineer(con_te_feature[i][col_name_rf], te_con_pred[i], con_te_feature[i]['Timestamp'])

    est1=RandomForestClassifier( n_jobs=-1,
        n_estimators =50,
        max_features='sqrt'
    ).fit(feature_tr, tr_label_g[i])

    pred_con_tr[i]=est1.predict(feature_con_tr)
    pred_con_te[i]=est1.predict(feature_con_te)
    tr_pred[i] = np.array([rg[i].grid_center[idx] for idx in pred_con_tr[i]])
    te_pred[i] = np.array([rg[i].grid_center[idx] for idx in pred_con_te[i]])
    error_tr[i] = [distance(pt1, pt2) for pt1, pt2 in zip(tr_pred[i], con_tr_p[i].values)]
    error_te[i] = [distance(pt1, pt2) for pt1, pt2 in zip(te_pred[i], con_te_p[i].values)]

In [20]:
for i in range(9):
    tr_feature_r[i]['Longitude'] = tr_label_[i].iloc[:, 0].values
    tr_feature_r[i]['Latitude'] = tr_label_[i].iloc[:, 1].values
    tr_feature_r[i]['gid'] = tr_label_g[i]

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documenta

In [21]:
conf_label_tr, conf_label_te = [[]]*9, [[]]*9

for i in range(9):
    conf_label_tr[i] = conf_model_label(error_tr[i])
    conf_label_te[i] = conf_model_label(error_te[i])

In [22]:
for i in range(9):
    con_tr_feature[i]["conf"]= conf_label_tr[i]
    con_te_feature[i]["conf"]= conf_label_te[i]
    con_tr_feature[i]["error"]= error_tr[i]
    con_te_feature[i]["error"]= error_te[i]

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documenta

In [23]:
for i in range(9):
    con_tr_feature[i]['Longitude'] = con_tr_p[i].iloc[:,0]
    con_tr_feature[i]['Latitude'] = con_tr_p[i].iloc[:,1]
    con_te_feature[i]['Longitude'] = con_te_p[i].iloc[:,0]
    con_te_feature[i]['Latitude'] = con_te_p[i].iloc[:,1]

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documenta

In [24]:
for i in range(9):
    con_tr_feature[i]['p_gid'] = pred_con_tr[i]
    con_tr_feature[i]['gid'] = con_tr_j[i]
    con_te_feature[i]['p_gid'] = pred_con_te[i]
    con_te_feature[i]['gid'] = con_te_j[i]

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documenta

In [25]:
def rss_level(dbm):
    if dbm>-50:
        return 1
    elif dbm >-60:
        return 2
    elif dbm >-70:
        return 3
    elif dbm>-80:
        return 4
    elif dbm>-90:
        return 5
    elif dbm>-100:
        return 6
    elif dbm>-110:
        return 7
    else:
        return 8

In [26]:
for i in range(9):
    for j in range(1, 8):
        con_tr_feature[i]['rss_level_%d' % j] = con_tr_feature[i]['Dbm_%d' % j].map(lambda x: rss_level(x))  
        con_te_feature[i]['rss_level_%d' % j] = con_te_feature[i]['Dbm_%d' % j].map(lambda x: rss_level(x))  

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [27]:
total_ob, total_ob_bs, total_ob_rss, total_ob_te = [[]]*9, [[]]*9, [[]]*9, [[]]*9, 

for i in range(9):
    total_ob[i] = con_tr_feature[i][['BSID','rss_level_1','BSID2','rss_level_2','BSID3','rss_level_3','BSID4','rss_level_4',
                   'BSID5','rss_level_5','BSID6','rss_level_6','BSID7','rss_level_7']].drop_duplicates()
    total_ob_bs[i] = con_tr_feature[i][['BSID','BSID2','BSID3','BSID4','BSID5','BSID6','BSID7']].drop_duplicates()
    total_ob_rss[i] = con_tr_feature[i][['rss_level_1','rss_level_2','rss_level_3','rss_level_4','rss_level_5','rss_level_6',
                               'rss_level_7']].drop_duplicates()
    total_ob_te[i] = con_te_feature[i][['BSID','rss_level_1','BSID2','rss_level_2','BSID3','rss_level_3','BSID4','rss_level_4',
               'BSID5','rss_level_5','BSID6','rss_level_6','BSID7','rss_level_7']].drop_duplicates()

In [28]:
def jaccard_sim(list1, list2):
    #print list1, list2
    union_set = len(set(list1)|set(list2))#并集长度
    intersection_set = len(set(list1)&set(list2))#交集长度

    Jaccard = float(intersection_set/union_set) #Jaccar
    return Jaccard

In [29]:
def adaptive_emission_pro(jd_list, match, bs_list, ss_list, can_list, total_c, conf, i):
    pro_list = []
    weight_list = []
    if match.shape[0]>0:
        weight_list.append(math.log(1+match.shape[0])*1)
        match_ss= match[(match['rss_level_1']== int(ss_list[0])) & (match['rss_level_2']==int(ss_list[1])) & (match['rss_level_3']==ss_list[2])
             & (match['rss_level_4']==ss_list[3]) & (match['rss_level_5']==ss_list[4]) & (match['rss_level_6']==ss_list[5]) & (match['rss_level_7']==ss_list[6])]
        if match_ss.shape[0]>0:
            pro_list.append(float(match_ss[match_ss['conf']==conf].shape[0])/float(total_c))
        else:
            pro_list.append(float(match[match['conf']==conf].shape[0])/float(total_c))
        
        
    for can_temp, jd in zip(can_list, jd_list):
        match_c = con_tr_feature[i][(con_tr_feature[i]['BSID']==int(can_temp[0])) & (con_tr_feature[i]['BSID2']==int(can_temp[1]))
                      &(con_tr_feature[i]['BSID3']==int(can_temp[2])) & (con_tr_feature[i]['BSID4']==int(can_temp[3]))
                      &(con_tr_feature[i]['BSID5']==int(can_temp[4])) & (con_tr_feature[i]['BSID6']==int(can_temp[5]))
                                 & (con_tr_feature[i]['BSID7']==int(can_temp[6]))]
        count = match_c.shape[0]
        weight_list.append(math.log(1+count)*jd)
        
        match_ss_c= match_c[(match_c['rss_level_1']== ss_list[0]) & (match_c['rss_level_2']== ss_list[1]) 
                            & (match_c['rss_level_3']==ss_list[2])& (match_c['rss_level_4']==ss_list[3]) 
                            & (match_c['rss_level_5']==ss_list[4]) & (match_c['rss_level_6']==ss_list[5])
                            & (match_c['rss_level_7']==ss_list[6])]
        if match_ss_c.shape[0]>0:
            pro_list.append(float(match_ss_c[match_ss_c['conf']==conf].shape[0])/float(total_c))
        else:
            pro_list.append(float(match_c[match_c['conf']==conf].shape[0])/float(total_c))


    weight_sum = np.sum(weight_list)
    ad_em_po = 0
    
    for x, y in zip(pro_list, weight_list):
        ad_em_po += x * (y / weight_sum)
    
    return ad_em_po

In [39]:
zeros_list = [[]]*9

for k in range(9):
    zeros_list[k] = []
    i =0
    j=0
    zero_num = con_tr_feature[k][con_tr_feature[k]['conf']==0].shape[0]
    print total_ob_te[k].shape
    for idx, row in total_ob_te[k].iterrows():
        bs_list =row[['BSID','BSID2','BSID3','BSID4','BSID5','BSID6','BSID7']].values
        ss_list = row[['rss_level_1','rss_level_2','rss_level_3','rss_level_4','rss_level_5','rss_level_6','rss_level_7']].values
        match = con_tr_feature[k][(con_tr_feature[k]['BSID']==int(bs_list[0])) & (con_tr_feature[k]['BSID2']==int(bs_list[1]))
                              &(con_tr_feature[k]['BSID3']==int(bs_list[2])) & (con_tr_feature[k]['BSID4']==int(bs_list[3]))
                              &(con_tr_feature[k]['BSID5']==int(bs_list[4])) & (con_tr_feature[k]['BSID6']==int(bs_list[5]))
                              &(con_tr_feature[k]['BSID7']==int(bs_list[6]))]
        if match.shape[0]<5:
            can_bs_row = []
            can_j = []
            jaccd_max = []
            idx_list = []
            for idxx, roww in total_ob_bs[k].iterrows():
                can_bs_list = roww.values
                jd = jaccard_sim(bs_list, can_bs_list)
                jaccd_max.append(jd)
                idx_list.append(idxx)
                if jd>0.5:
                    can_bs_row.append(can_bs_list)
                    can_j.append(jd)

            if len(can_bs_row)==0:
                can_idx = jaccd_max.index(np.max(jaccd_max))
                can_temp = total_ob_bs[k].iloc[can_idx,:].values
                can_bs_row.append(total_ob_bs[k].iloc[can_idx,:].values)
                match_c = con_tr_feature[k][(con_tr_feature[k]['BSID']==int(can_temp[0])) & (con_tr_feature[k]['BSID2']==int(can_temp[1]))
                              &(con_tr_feature[k]['BSID3']==int(can_temp[2])) & (con_tr_feature[k]['BSID4']==int(can_temp[3]))
                              &(con_tr_feature[k]['BSID5']==int(can_temp[4])) & (con_tr_feature[k]['BSID6']==int(can_temp[5]))
                              &(con_tr_feature[k]['BSID7']==int(can_temp[6]))]

                match_ss_c= match_c[(match_c['rss_level_1']== ss_list[0]) & (match_c['rss_level_2']== ss_list[1]) 
                                & (match_c['rss_level_3']==ss_list[2])& (match_c['rss_level_4']==ss_list[3]) 
                                & (match_c['rss_level_5']==ss_list[4]) & (match_c['rss_level_6']==ss_list[5])
                                & (match_c['rss_level_7']==ss_list[6])]
                if match_ss_c.shape[0]>0:
                    zeros_list[k].append(float(match_ss_c[match_ss_c['conf']==0].shape[0])/float(zero_num))
                else:
                    zeros_list[k].append(float(match_c[match_c['conf']==0].shape[0])/float(zero_num))

                j+=1 
            else:
                zeros_list[k].append(adaptive_emission_pro(can_j, match, bs_list, ss_list, can_bs_row, zero_num, 0, k))
            #print i, len(can_bs_row)
            i+=1
        else:
            j+=1
            match_ss= match[(match['rss_level_1']== int(ss_list[0])) & (match['rss_level_2']==int(ss_list[1])) & (match['rss_level_3']==ss_list[2])
                 & (match['rss_level_4']==ss_list[3]) & (match['rss_level_5']==ss_list[4]) & (match['rss_level_6']==ss_list[5]) & (match['rss_level_7']==ss_list[6])]
            if match_ss.shape[0]>0:
                zeros_list[k].append(float(match_ss[match_ss['conf']==0].shape[0])/float(zero_num))
            else:
                zeros_list[k].append(float(match[match['conf']==0].shape[0])/float(zero_num))

(2647, 14)
(2039, 14)
(620, 14)
(1251, 14)
(2551, 14)
(2488, 14)
(2735, 14)
(2057, 14)
(353, 14)


In [42]:
zeros_list2 = [[]]*9

for i in range(9):
#     zeros_list2[i] = np.array(zeros_list[i])
#     np.savetxt('zeros_list'+str(i)+'.txt', zeros_list2[i])
    zeros_list2[i] = np.loadtxt('zeros_list'+str(i)+'.txt')
    zeros_list2[i] = zeros_list2[i].tolist()

In [43]:
for i in range(9):
    print total_ob_te[i].shape, len(zeros_list2[i])
    total_ob_te[i]['conf_ad_em_pro_0'] = zeros_list2[i]

(2647, 14) 2647
(2039, 14) 2039
(620, 14) 620
(1251, 14) 1251
(2551, 14) 2551
(2488, 14) 2488
(2735, 14) 2735
(2057, 14) 2057
(353, 14) 353


In [46]:
one_list = [[]]*9

for i in range(9):
    one_list[i] = []
    one_num = con_tr_feature[i][con_tr_feature[i]['conf']==1].shape[0]
    for idx, row in total_ob_te[i].iterrows():
        bs_list =row[['BSID','BSID2','BSID3','BSID4','BSID5','BSID6','BSID7']].values
        ss_list = row[['rss_level_1','rss_level_2','rss_level_3','rss_level_4','rss_level_5','rss_level_6','rss_level_7']].values
        #print bs_list[0]
        match = con_tr_feature[i][(con_tr_feature[i]['BSID']==int(bs_list[0])) & (con_tr_feature[i]['BSID2']==int(bs_list[1]))
                              &(con_tr_feature[i]['BSID3']==int(bs_list[2])) & (con_tr_feature[i]['BSID4']==int(bs_list[3]))
                              &(con_tr_feature[i]['BSID5']==int(bs_list[4])) & (con_tr_feature[i]['BSID6']==int(bs_list[5]))
                              &(con_tr_feature[i]['BSID7']==int(bs_list[6]))]
        if match.shape[0]<5:
            can_bs_row = []
            can_j = []
            jaccd_max = []
            idx_list = []
            for idxx, roww in total_ob_bs[i].iterrows():
                can_bs_list = roww.values
                jd = jaccard_sim(bs_list, can_bs_list)
                jaccd_max.append(jd)
                idx_list.append(idxx)
                if jd>0.5:
                    can_bs_row.append(can_bs_list)
                    can_j.append(jd)

            if len(can_bs_row)==0:
                can_idx = jaccd_max.index(np.max(jaccd_max))
                can_temp = total_ob_bs[i].iloc[can_idx,:].values
                can_bs_row.append(total_ob_bs[i].iloc[can_idx,:].values)
                match_c = con_tr_feature[i][(con_tr_feature[i]['BSID']==int(can_temp[0])) & (con_tr_feature[i]['BSID2']==int(can_temp[1]))
                              &(con_tr_feature[i]['BSID3']==int(can_temp[2])) & (con_tr_feature[i]['BSID4']==int(can_temp[3]))
                              &(con_tr_feature[i]['BSID5']==int(can_temp[4])) & (con_tr_feature[i]['BSID6']==int(can_temp[5]))
                              &(con_tr_feature[i]['BSID7']==int(can_temp[6]))]

                match_ss_c= match_c[(match_c['rss_level_1']== ss_list[0]) & (match_c['rss_level_2']== ss_list[1]) 
                                & (match_c['rss_level_3']==ss_list[2])& (match_c['rss_level_4']==ss_list[3]) 
                                & (match_c['rss_level_5']==ss_list[4]) & (match_c['rss_level_6']==ss_list[5])
                                & (match_c['rss_level_7']==ss_list[6])]
                if match_ss_c.shape[0]>0:
                    one_list[i].append(float(match_ss_c[match_ss_c['conf']==1].shape[0])/float(one_num))
                else:
                    one_list[i].append(float(match_c[match_c['conf']==1].shape[0])/float(one_num))

            else:
                one_list[i].append(adaptive_emission_pro(can_j, match, bs_list, ss_list, can_bs_row, one_num, 1, i))

        else:

            match_ss= match[(match['rss_level_1']== int(ss_list[0])) & (match['rss_level_2']==int(ss_list[1])) & (match['rss_level_3']==ss_list[2])
                 & (match['rss_level_4']==ss_list[3]) & (match['rss_level_5']==ss_list[4]) & (match['rss_level_6']==ss_list[5]) & (match['rss_level_7']==ss_list[6])]
            if match_ss.shape[0]>0:
                one_list[i].append(float(match_ss[match_ss['conf']==1].shape[0])/float(one_num))
            else:
                one_list[i].append(float(match[match['conf']==1].shape[0])/float(one_num))

In [47]:
one_list2 = [[]]*9

for i in range(9):
#     one_list2[i] = np.array(one_list[i])
#     np.savetxt('one_list'+str(i)+'.txt', one_list2[i])
    one_list2[i] = np.loadtxt('one_list'+str(i)+'.txt')
    one_list2[i] = one_list2[i].tolist()

In [48]:
for i in range(9):
    total_ob_te[i]['conf_ad_em_pro_1'] = one_list2[i]

In [49]:
for i in range(9):
    con_te_feature[i]['gid'] = con_te_j[i]
    con_te_feature[i]['p_gid'] = pred_con_te[i]
    con_tr_feature[i]['gid'] = con_tr_j[i]
    con_tr_feature[i]['p_gid'] = pred_con_tr[i]

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documenta

In [50]:
trajs_tr = [[]]*9

for i in range(9):
    trajs_tr[i] = con_tr_feature[i].groupby(['TrajID'])

In [51]:
time_list, st_mat = [[]]*9, [[]]*9

for k in range(9):
    time_list[k] = []
    st_mat[k] = np.zeros((14,2,2)) #0-0 0-1 1-0, 1-1
    for trajid, traj in trajs_tr[k]:
        traj = traj.sort_values(by=['Timestamp'],ascending=True)
        t_time = traj['Timestamp'].values
        conf = traj['conf'].values
        for i in range(traj.shape[0]-1):
            time_list[k].append(compute_time_interval(t_time[i], t_time[i + 1]))
            idx = int(compute_time_interval(t_time[i], t_time[i + 1])/5)
            if idx >12:
                idx = 13
            if conf[i]==0 and conf[i+1]==0:
                st_mat[k][idx, 0, 0] +=1
            if conf[i] ==0 and conf[i+1] ==1:
                st_mat[k][idx, 0, 1] +=1
            if conf[i]==1 and conf[i+1]==0:
                st_mat[k][idx, 1, 0] +=1
            if conf[i] ==1 and conf[i+1] ==1:
                st_mat[k][idx, 1, 1] +=1

In [52]:
trajs_te = [[]]*9

for i in range(9):
    trajs_te[i] = con_te_feature[i].groupby(['TrajID'])

In [53]:
pred_list_t_idx, pred_list, p_g_list, t_g_list = [[]]*9, [[]]*9, [[]]*9, [[]]*9

for k in range(9):
    pred_list_t_idx[k] = []
    #st_mat = np.zeros((14,2,2)) #0-0 0-1 1-0, 1-1
    pred_list[k] = []
    p_g_list[k] = []
    t_g_list[k] = []
    for trajid, traj in trajs_te[k]:
        traj = traj.sort_values(by=['Timestamp'],ascending=True)
        t_time = traj['Timestamp'].values
        conf = traj['conf'].values
        idx_list = []
        for i in range(traj.shape[0]-1):
            time_list.append(compute_time_interval(t_time[i], t_time[i + 1]))
            idx = int(compute_time_interval(t_time[i], t_time[i + 1])/5)
            if idx >12:
                idx = 13
            idx_list.append(idx)
        pred_list[k].append(traj[['BSID','rss_level_1','BSID2','rss_level_2',
                   'BSID3','rss_level_3','BSID4','rss_level_4',
                   'BSID5','rss_level_5','BSID6','rss_level_6',]].values)

        pred_list_t_idx[k].append(idx_list)
        p_g_list[k].append(traj['p_gid'].values)
        t_g_list[k].append(traj[['Longitude','Latitude']].values)

In [54]:
init_prob = [[]]*9

for i in range(9):
    init_prob[i] = [float(con_tr_feature[i][con_tr_feature[i]['conf']==0].shape[0])/con_tr_feature[i].shape[0],
                 float(con_tr_feature[i][con_tr_feature[i]['conf']==1].shape[0])/con_tr_feature[i].shape[0]]

In [55]:
def hmm_viterbi(st_mat, init_prob, emit_prob, obs_seq, t_list):
    Nstate = 2
    Nobs = int(emit_prob.shape[0])
    T = len(obs_seq)
    
    partial_prob = np.zeros((Nstate,T))

    path = np.zeros((Nstate,T))

    for i in range(Nstate):
        partial_prob[i,0] = init_prob[i] * emit_prob[obs_seq[0], i]
        path[i,0] = i


    for t in range(1,T,1):
        newpath = np.zeros((Nstate,T))
        for i in range(Nstate):
            prob = -1.0
            for j in range(Nstate):
                nprob = partial_prob[j,t-1] * st_mat[t_list[t-1], j, i] * emit_prob[obs_seq[t], i]
                if nprob > prob:
                    prob = nprob
                    partial_prob[i,t] = nprob
                    newpath[i,0:t] = path[j,0:t]
                    newpath[i,t] = i
        path = newpath
    
    prob = -1.0
    j = 0
    for i in range(Nstate):
        if(partial_prob[i,T-1] > prob):
            prob = partial_prob[i,T-1]
            j = i

    return path[j,:]

In [56]:
em_pro = [[]]*9

for i in range(9):
    em_pro[i] = total_ob_te[i]
    em_pro[i] = em_pro[i].reset_index(drop = True)

In [57]:
emit_prob = [[]]*9

for i in range(9):
    emit_prob[i] = em_pro[i][['conf_ad_em_pro_0', 'conf_ad_em_pro_1']].values

In [63]:
w, tj_r, standard_point, g_list = [[]]*9, [[]]*9, [[]]*9, [[]]*9

for i in range(9):
    num_g = rg[i].n_grid
    w[i] = np.zeros((num_g, num_g))
    
    test = zip(pred_con_tr[i], con_tr_j[i], error_tr[i])
    tj_r[i] = {}
    for ss in test:
        if ss[0]!=ss[1] and ss[2]>50:
             w[i][ss[0]][ss[1]] +=1
        if ss[0] not in tj_r[i]:
            tj_r[i][ss[0]]=set()
        tj_r[i][ss[0]].add((ss[1]))

    standard_point[i]=[]


    for item in tj_r[i]:
        if (len(tj_r[i][item])==1) and (item in tj_r[i][item]):
            standard_point[i].append(item)

    for idx in range(num_g):
        if idx not in tj_r[i]:
            tj_r[i][idx]=set()
            tj_r[i][idx].add((idx))

    g_list[i]=rg[i].gridlist

In [64]:
g_obs = [[]]*9

for i in range(9):
    g_obs[i] = {}
    for idx, row in con_tr_feature[i].iterrows():
        obs = row[['BSID','rss_level_1','BSID2','rss_level_2',
                   'BSID3','rss_level_3','BSID4','rss_level_4',
                   'BSID5','rss_level_5','BSID6','rss_level_6',]].values
        gid = int(row['gid'])
        match = em_pro[i][(em_pro[i]['BSID']==obs[0]) & (em_pro[i]['rss_level_1']==obs[1]) & 
                          (em_pro[i]['BSID2']==obs[2]) & (em_pro[i]['rss_level_2']==obs[3]) & 
                          (em_pro[i]['BSID3']==obs[4]) & (em_pro[i]['rss_level_3']==obs[5]) & 
                          (em_pro[i]['BSID4']==obs[6]) & (em_pro[i]['rss_level_4']==obs[7]) & 
                          (em_pro[i]['BSID5']==obs[8]) & (em_pro[i]['rss_level_5']==obs[9]) & 
                          (em_pro[i]['BSID6']==obs[10]) & (em_pro[i]['rss_level_6']==obs[11])]
        if match.shape[0]>0:
            obs_idx = (gid, int(match.index[0]))
            if obs_idx not in g_obs[i]:
                g_obs[i][obs_idx]=1
            else:
                g_obs[i][obs_idx] +=1

In [67]:
def repair(te_pred, cof_list, o_list, k):
    i=0
    te_pred_n=[]
    while i < len(cof_list):
        if cof_list[i]==0:
            pred_temp = te_pred[i]
            if pred_temp in standard_point[k]:
                te_pred_n.append(te_pred[i])
            else:
                repair_r = list(w[k][te_pred[i], :])
                max_idx = repair_r.index(np.max(repair_r))
                if (max_idx, o_list[i]) in g_obs[k]:
                    te_pred_n.append(int(max_idx))
                    #print 'ok'
                else:
                    
                    te_pred_n.append(te_pred[i])
                
        else:
            te_pred_n.append(te_pred[i])
        i+=1
            
    #print len(te_pred_n)
    return te_pred_n

In [80]:
import operator
error = []

for i in range(9):
    error_new =[]
    for row, time, raw, gt in zip (pred_list[i], pred_list_t_idx[i], p_g_list[i], t_g_list[i]):

        o_seq = []
        for obs in row:
            #print obs
            match = em_pro[i][(em_pro[i]['BSID']==obs[0]) & (em_pro[i]['rss_level_1']==obs[1]) & 
                          (em_pro[i]['BSID2']==obs[2]) & (em_pro[i]['rss_level_2']==obs[3]) & 
                          (em_pro[i]['BSID3']==obs[4]) & (em_pro[i]['rss_level_3']==obs[5]) & 
                          (em_pro[i]['BSID4']==obs[6]) & (em_pro[i]['rss_level_4']==obs[7]) & 
                          (em_pro[i]['BSID5']==obs[8]) & (em_pro[i]['rss_level_5']==obs[9]) & 
                          (em_pro[i]['BSID6']==obs[10]) & (em_pro[i]['rss_level_6']==obs[11])]
            o_seq.append(match.index[0])
        pred = raw
        true = gt
        cof_list = hmm_viterbi(st_mat[i], init_prob[i], emit_prob[i], o_seq, time)
        r_pred = repair(pred, cof_list, o_seq, i)


        te_predp = np.array([rg[i].grid_center[idx] for idx in r_pred])

        error_tep = [distance(pt1, pt2) for pt1, pt2 in zip(te_predp, true)]

        for t in error_tep:
            error_new.append(t)  
    print np.median(error_new)
    error += error_new

684.5
635.0
447.0
778.35
379.0
605.1
421.65
626.2
895.7


In [69]:
err= sorted(error)

In [70]:
print(np.median(error), np.mean(error), err[int(len(err) * 0.9)])

(548.70000000000005, 599.79654846782194, 1097.0)
